In [2]:
# Chapter 7.2 - 퓨샷 프롬프트와 예시 선택기 (최신 버전)
# 원서 코드의 LLMChain 및 chain.run() 메소드는 deprecated 되었습니다.

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

print("OpenAI API 키 설정이 필요합니다.")
print("API 키 발급: https://platform.openai.com/api-keys")

# API 키가 이미 환경 변수에 있는지 확인
if 'OPENAI_API_KEY' not in os.environ:
    print("\n다음 중 한 가지 방법을 선택하세요:")
    print("1. 아래에서 직접 입력 (안전함 - 화면에 표시되지 않음)")
    print("2. 환경 변수로 설정 (export OPENAI_API_KEY=your-key)")
    print("3. .env 파일 사용")
    
    # 안전한 입력 방법
    try:
        api_key = getpass.getpass("\nAPI 키를 입력하세요 (입력 시 화면에 표시되지 않음): ")
        if api_key:
            os.environ['OPENAI_API_KEY'] = api_key
            print("✅ API 키가 안전하게 설정되었습니다!")
        else:
            print("⚠️ API 키가 입력되지 않았습니다.")
    except KeyboardInterrupt:
        print("\n입력이 취소되었습니다.")
else:
    print("✅ 기존 환경 변수의 API 키를 사용합니다.")

print("\n" + "="*70)

# =============================================================================
# 라이브러리 import 및 LLM 설정
# =============================================================================
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print("🤖 LLM 모델 초기화 완료")

# =============================================================================
# 퓨샷 프롬프트 예제 1: 동물 서식지 분류
# =============================================================================
print("\n" + "="*70)
print("📚 퓨샷 프롬프트 예제 1: 동물 서식지 분류")
print("="*70)

# 예시 데이터 (Few-shot examples)
examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

# 각 예시를 포맷하는 템플릿
example_template = """Animal: {animal}
Habitat: {habitat}"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

# FewShotPromptTemplate 정의
fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal.\n(주어진 동물의 서식지를 확인해줘.)",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n"
)

print("🔍 생성된 프롬프트 구조 확인:")
sample_formatted = fewshot_prompt.format(input="tiger")
print(sample_formatted)
print("-" * 50)

# =============================================================================
# 방법 1: 최신 RunnableSequence 방식 (권장)
# =============================================================================
print("\n방법 1: RunnableSequence 사용 (LangChain 0.2+ 권장)")

# 최신 방식: prompt | llm 파이프라인
chain = fewshot_prompt | llm

# 테스트 실행
test_animals = ["tiger", "dolphin", "kangaroo", "penguin"]

try:
    for animal in test_animals:
        response = chain.invoke({"input": animal})
        print(f"🐾 {animal}: {response.content}")
        
except Exception as e:
    print(f"⚠️ 오류 발생: {e}")
    print("💡 API 키가 정확히 설정되었는지 확인하세요.")

# =============================================================================
# 방법 2: 기존 방식과 비교 (참고용)
# =============================================================================
print("\n" + "="*70)
print("참고: 기존 방식 vs 새로운 방식")
print("="*70)

print("""
# 이전 방식 (Deprecated):
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=fewshot_prompt)  # ⚠️ Deprecated
response = chain.run({"input": "tiger"})          # ⚠️ Deprecated

# 새로운 방식 (권장):
chain = fewshot_prompt | llm                      # ✅ 권장
response = chain.invoke({"input": "tiger"})       # ✅ 권장
""")

# =============================================================================
# 퓨샷 프롬프트 예제 2: 감정 분류
# =============================================================================
print("\n" + "="*70)
print("📚 퓨샷 프롬프트 예제 2: 감정 분류")
print("="*70)

# 감정 분류 예시 데이터
emotion_examples = [
    {"text": "I love this new restaurant!", "emotion": "positive"},
    {"text": "The service was terrible and slow.", "emotion": "negative"},
    {"text": "The weather is okay today.", "emotion": "neutral"}
]

# 감정 분류용 예시 템플릿
emotion_example_template = """Text: {text}
Emotion: {emotion}"""

emotion_example_prompt = PromptTemplate(
    input_variables=["text", "emotion"],
    template=emotion_example_template
)

# 감정 분류용 FewShotPromptTemplate
emotion_fewshot_prompt = FewShotPromptTemplate(
    examples=emotion_examples,
    example_prompt=emotion_example_prompt,
    prefix="Classify the emotion of the given text as positive, negative, or neutral.\n(주어진 텍스트의 감정을 긍정, 부정, 중립으로 분류하세요.)",
    suffix="Text: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n\n"
)

# 감정 분류 체인
emotion_chain = emotion_fewshot_prompt | llm

# 테스트 문장들
test_sentences = [
    "This movie is absolutely amazing!",
    "I'm disappointed with the quality.",
    "It's an average product.",
    "이 음식이 정말 맛있어요!",
    "서비스가 별로였습니다."
]

try:
    print("💭 감정 분류 결과:")
    for sentence in test_sentences:
        result = emotion_chain.invoke({"input": sentence})
        print(f"'{sentence}' → {result.content}")
        
except Exception as e:
    print(f"⚠️ 감정 분류 중 오류: {e}")

# =============================================================================
# 퓨샷 프롬프트 저장 및 로드
# =============================================================================
print("\n" + "="*70)
print("💾 프롬프트 저장 및 로드")
print("="*70)

# 프롬프트를 JSON 파일로 저장
try:
    fewshot_prompt.save("animal_habitat_prompt.json")
    print("✅ 프롬프트가 'animal_habitat_prompt.json'에 저장되었습니다.")
    
    # 저장된 프롬프트 다시 로드
    from langchain.prompts import load_prompt
    loaded_prompt = load_prompt("animal_habitat_prompt.json")
    print("✅ 프롬프트가 성공적으로 로드되었습니다.")
    
    # 로드된 프롬프트 테스트
    loaded_chain = loaded_prompt | llm
    test_result = loaded_chain.invoke({"input": "shark"})
    print(f"🦈 로드된 프롬프트 테스트 - shark: {test_result.content}")
    
except Exception as e:
    print(f"⚠️ 저장/로드 중 오류: {e}")

# =============================================================================
# 고급 활용: 동적 예시 선택
# =============================================================================
print("\n" + "="*70)
print("🔧 고급 활용 팁")
print("="*70)

print("""
1. 예시 개수 제한:
   FewShotPromptTemplate(
       examples=examples[:2],  # 처음 2개 예시만 사용
       max_length=1000,        # 최대 길이 제한
       ...
   )

2. 동적 예시 선택:
   from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
   from langchain.vectorstores import Chroma
   from langchain.embeddings import OpenAIEmbeddings
   
   # 의미적 유사성 기반 예시 선택기
   example_selector = SemanticSimilarityExampleSelector.from_examples(
       examples, OpenAIEmbeddings(), Chroma, k=2
   )

3. 조건부 예시:
   # 입력에 따라 다른 예시 사용
   def select_examples(input_text):
       if "marine" in input_text.lower():
           return marine_examples
       else:
           return land_examples

4. 템플릿 체인:
   # 여러 퓨샷 프롬프트를 연결
   chain1 = prompt1 | llm | output_parser
   chain2 = prompt2 | llm
   final_chain = chain1 | chain2
""")

print("\n🎉 Chapter 7.2 퓨샷 프롬프트 예제 완료!")
print("📚 다음 단계: 7.3절 LangChain에서 체인이란")

OpenAI API 키 설정이 필요합니다.
API 키 발급: https://platform.openai.com/api-keys
✅ 기존 환경 변수의 API 키를 사용합니다.

🤖 LLM 모델 초기화 완료

📚 퓨샷 프롬프트 예제 1: 동물 서식지 분류
🔍 생성된 프롬프트 구조 확인:
Identify the habitat of the given animal.
(주어진 동물의 서식지를 확인해줘.)

Animal: lion
Habitat: savanna

Animal: polar bear
Habitat: Arctic ice

Animal: elephant
Habitat: African grasslands

Animal: tiger
Habitat:
--------------------------------------------------

방법 1: RunnableSequence 사용 (LangChain 0.2+ 권장)
🐾 tiger: rainforest or grasslands
🐾 dolphin: ocean
🐾 kangaroo: Australian outback
🐾 penguin: Antarctic ice shelves

참고: 기존 방식 vs 새로운 방식

# 이전 방식 (Deprecated):
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=fewshot_prompt)  # ⚠️ Deprecated
response = chain.run({"input": "tiger"})          # ⚠️ Deprecated

# 새로운 방식 (권장):
chain = fewshot_prompt | llm                      # ✅ 권장
response = chain.invoke({"input": "tiger"})       # ✅ 권장


📚 퓨샷 프롬프트 예제 2: 감정 분류
💭 감정 분류 결과:
'This movie is absolutely amazing!' → posit

No `_type` key found, defaulting to `prompt`.


'서비스가 별로였습니다.' → negative

💾 프롬프트 저장 및 로드
✅ 프롬프트가 'animal_habitat_prompt.json'에 저장되었습니다.
✅ 프롬프트가 성공적으로 로드되었습니다.
🦈 로드된 프롬프트 테스트 - shark: ocean

🔧 고급 활용 팁

1. 예시 개수 제한:
   FewShotPromptTemplate(
       examples=examples[:2],  # 처음 2개 예시만 사용
       max_length=1000,        # 최대 길이 제한
       ...
   )

2. 동적 예시 선택:
   from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
   from langchain.vectorstores import Chroma
   from langchain.embeddings import OpenAIEmbeddings
   
   # 의미적 유사성 기반 예시 선택기
   example_selector = SemanticSimilarityExampleSelector.from_examples(
       examples, OpenAIEmbeddings(), Chroma, k=2
   )

3. 조건부 예시:
   # 입력에 따라 다른 예시 사용
   def select_examples(input_text):
       if "marine" in input_text.lower():
           return marine_examples
       else:
           return land_examples

4. 템플릿 체인:
   # 여러 퓨샷 프롬프트를 연결
   chain1 = prompt1 | llm | output_parser
   chain2 = prompt2 | llm
   final_chain = chain1 | chain2


🎉 Chapter 7.2 퓨샷 프롬프트 예제 완료!
📚